<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.1: Your First Chisel Module
**Prev: [Introduction to Scala](1_intro_to_scala.ipynb)**<br>
**Next: [Combinational Logic](2.2_comb_logic.ipynb)**

## Motivation
이제 Scala에 익숙해졌으므로 하드웨어 조각을 시작하겠습니다! Chisel은 **C**onstructing **H**ardware **I**n a **S**cala **E**mbedded **L**anguage를 나타냅니다. 즉, Scala의 DSL이므로 동일한 코드 내에서 Scala와 Chisel 프로그래밍을 모두 활용할 수 있습니다. 어떤 코드가 "Scala"이고 어떤 코드가 "Chisel"인지 이해하는 것이 중요하지만 나중에 더 자세히 설명하겠습니다. 지금은 Chisel과 모듈 2의 코드를 Verilog를 작성하는 더 나은 방법으로 생각하십시오. 이 모듈은 전체 Chisel `Module`과 tester를 사용자에게 던집니다. 지금은 요점만 파악하세요. 나중에 더 많은 예제를 볼 수 있습니다.

## Setup
다음 셀은 Chisel에 필요한 종속성을 다운로드합니다. 미래의 모든 notebook에서 볼 수 있습니다. **지금 본 셀을 실행 하십시오**.

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp/source/load-ivy.sc"

마지막 모듈에서 언급했듯이 이러한 명령문은 Chisel을 가져오기 위해 필요합니다. 미래의 코드 블록을 실행하기 전에 **본 셀을 실행**하십시오.

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

---
# Your First Module
이 섹션에서는 첫 번째 하드웨어 모듈, 테스트 사례 및 실행 방법을 설명합니다. 그것은 당신이 이해하지 못할 많은 것들을 포함할 것이고 그것은 괜찮습니다. 우리는 당신이 배운 내용을 강화하기 위해 이 완전하고 실제적인 예제로 계속해서 되돌아갈 수 있도록 광범위한 스트로크를 제거하기를 바랍니다.

<span style="color:blue">**Example: A Module**</span><br>
Verilog와 마찬가지로 Chisel에서 모듈 정의를 선언할 수 있습니다. 다음 예는 하나의 4비트 입력 `in`과 하나의 4비트 출력 `out`이 있는 Chisel `Module`, `Passthrough`입니다. 모듈은 `in`과 `out`을 조합하여 연결하므로 `in`은 `out`을 구동합니다.

In [3]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

defined class Passthrough

다음은 우리가 기술하는 하드웨어 측면에서 각 라인을 생각하는 방법을 설명합니다.

```scala
class Passthrough extends Module {
```
`Passthrough`라는 새 모듈을 선언합니다. `Module`은 모든 하드웨어 모듈이 확장해야 하는 빌트인 Chisel 클래스입니다.


```scala 
val io = IO(...)
```
우리는 모든 입력 및 출력 포트를 특별한 `io` `val`로 선언합니다. 'io'라고 하고 'IO' 객체 또는 인스턴스여야 하며, `IO(_instantiated_bundle_)` 형식이 필요합니다.


```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```
각각 입력 및 출력 방향과 함께 `in` 및 `out`이라는 명명된 신호를 포함하는 새 하드웨어 구조체 유형(Bundle)을 선언합니다.


```scala
UInt(4.W)
```
신호의 하드웨어 유형을 선언합니다. 이 경우 너비가 4인 부호 없는 정수입니다.


```scala
io.out := io.in
```
입력 포트를 출력 포트에 연결하여 `io.in` *drive* `io.out`이 되도록 합니다. `:=` 연산자는 오른쪽 신호가 왼쪽 신호를 구동함을 나타내는 ***Chisel*** 연산자이며 방향 연산자입니다.

hardware construction languages(HCL)의 장점은 기본 프로그래밍 언어를 스크립팅 언어로 사용할 수 있다는 것입니다. 예를 들어, Chisel 모듈을 선언한 후 Scala를 사용하여 Chisel 컴파일러를 호출하여 Chisel `Passthrough`를 Verilog `Passthrough`로 변환합니다. 이 프로세스를 ***elaboration***라고 합니다.

In [4]:
// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd2.sc 6:10]
endmodule



<span style="color:blue">**Example: A Module Generator**</span><br>
Scala에 대해 배운 것을 이 예제에 적용하면 Chisel 모듈이 Scala 클래스로 구현되어 있음을 알 수 있습니다. 다른 Scala 클래스와 마찬가지로 Chisel 모듈이 몇 가지 구성 매개변수를 사용하도록 만들 수 있습니다. 이 경우 입력 및 출력 포트의 너비를 지정하는 정수 `width`를 허용하는 새 클래스 `PassthroughGenerator`를 만듭니다.

In [5]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// Let's now generate modules with different widths
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  output [9:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 6:10]
endmodule

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input         clock,
  input         reset,
  input  [19:0] io_in,
  output [19:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 6:10]
endmodule



defined class PassthroughGenerator

생성된 Verilog는 `width` 매개변수에 할당된 값에 따라 입/출력에 다른 비트폭을 사용합니다. 이것이 어떻게 작동하는지 알아봅시다. Chisel 모듈은 일반 Scala 클래스이기 때문에 Scala의 클래스 생성자의 힘을 사용하여 설계의 정교함을 매개변수화할 수 있습니다.

이 매개변수화는 *Chisel*이 아니라 *Scala*에 의해 활성화됨을 알 수 있습니다. Chisel에는 매개변수화를 위한 추가 API가 없지만 설계자는 단순히 Scala 기능을 활용하여 설계를 매개변수화할 수 있습니다.

`PassthroughGenerator`는 더 이상 단일 모듈을 설명하지 않고 대신 `width`로 매개변수화된 모듈 제품군을 설명하므로 이 `Passthrough`를 ***generator***라고 합니다.

---
# Testing Your Hardware

테스터 없이 하드웨어 모듈이나 생성기가 완성되어서는 안 됩니다. Chisel에는 이 부트캠프 전체에서 탐색할 테스트 기능이 내장되어 있습니다. 다음 예는 `Passthrough`의 입력 포트 `in` 인스턴스에 값을 전달하고 출력 포트 `out`에 동일한 값이 표시되는지 확인하는 Chisel 테스트 하네스입니다.


<span style="color:blue">**Example: A Tester**</span><br>

여기에는 고급 Scala가 있습니다. 그러나 `poke` 및 `expect` 명령 외에는 아무것도 이해할 필요가 없습니다. 나머지 코드는 이러한 간단한 테스트를 작성하기 위한 단순한 상용구로 생각할 수 있습니다.

In [6]:
// Scala Code: `test` runs the unit test. 
// test takes a user Module and has a code block that applies pokes and expects to the 
// circuit under test (c)
test(new Passthrough()) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


Elaborating design...
Done elaborating.
test Passthrough Success: 0 tests passed in 2 cycles in 0.018198 seconds 109.90 Hz
SUCCESS!!


무슨 일이야? 테스트는 `Passthrough` 모듈을 수락하고 모듈의 입력에 값을 할당하고 출력을 확인합니다. 입력을 설정하기 위해 `poke`를 호출합니다. 출력을 확인하기 위해 `expect`를 호출합니다. 출력을 예상 값과 비교하고 싶지 않다면(어설션 없음) 대신 출력을 `peek`할 수 있습니다.


모든 `expect` 문이 참이면 상용구 코드는 통과를 반환합니다.

>`poke` 및 `expect`는 Chisel 하드웨어 리터럴 표기법을 사용합니다. 두 작업 모두 올바른 유형의 리터럴을 예상합니다.
`UInt()`를 `포킹`하는 경우 `UInt` 리터럴(예: `c.io.in.poke(10.U)`, 마찬가지로 입력이 `Bool()`인 경우 ` poke`는 `true.B` 또는 `false.B`를 예상합니다.



<span style="color:red">**Exercise: Writing Your Own Testers**</span><br>
두 개의 테스트를 작성하고 실행합니다. 하나는 너비 10에 대해 `PassthroughGenerator`를 테스트하고 다른 하나는 너비 20에 대해 `PassthroughGenerator`를 테스트합니다. 각각에 대해 최소 두 개의 값인 0과 지정된 너비가 지원하는 최대값을 확인합니다. 세 개의 물음표는 스칼라에서 특별한 의미를 가지고 있습니다. 이 부트캠프 연습에서 자주 볼 수 있습니다. `???`로 코드를 실행하면 `NotImplementedError`가 생성됩니다. `???`를 자신의 코드로 바꿉니다.

In [ ]:
// Test with width 10

test(???) { c =>
    ???
}

// Test with width 20

test(???) { c =>
    ???
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong> (click to toggle displaying it)</label>
<article>
<pre style="background-color:#f7f7f7">
test(new PassthroughGenerator(10)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1023.U)
    c.io.out.expect(1023.U)
}

test(new PassthroughGenerator(20)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1048575.U)
    c.io.out.expect(1048575.U)
}

</pre></article></div></section></div>

---
# Looking at Generated Verilog/FIRRTL

생성된 하드웨어를 이해하는 데 문제가 있고 구조적 Verilog 및/또는 FIRRTL(Verilog의 합성 전용 하위 집합과 비교되는 Chisel의 IR)을 읽는 데 익숙하다면 생성된 Verilog를 보고 Chisel 실행 결과를 알 수 있습니다.

다음은 Verilog(이미 본)와 FIRRTL을 생성하는 예입니다.

In [7]:
// Viewing the Verilog for debugging
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd2.sc 6:10]
endmodule



In [8]:
// Viewing the firrtl for debugging
println(getFirrtl(new Passthrough))

Elaborating design...
Done elaborating.
circuit Passthrough :
  module Passthrough :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<4>, out : UInt<4>}

    io.out <= io.in @[cmd2.sc 6:10]



---
# You're done!

[Return to the top.](#top)

## <span style="color:red"> Appendix: A Note on "printf" Debugging</span>
[print 문으로 디버깅](https://stackoverflow.com/a/189570)이 항상 최선의 디버깅 방법은 아니지만 예상한 대로 작동하지 않을 때 무슨 일이 일어나고 있는지 확인하는 쉬운 첫 번째 단계인 경우가 많습니다.
Chisel generator는 하드웨어를 생성하는 프로그램이기 때문에 print 생성기 및 회로 상태에 대한 몇 가지 추가 미묘함이 있습니다.
인쇄 문이 언제 실행되고 무엇을 인쇄하는지 기억하는 것이 중요합니다.
인쇄할 수 있는 세 가지 일반적인 시나리오에는 몇 가지 중요한 차이점이 있습니다.
* 회로 생성 중 Chisel generator print
* 회로 시뮬레이션 중 회로 print
* 테스터는 테스트 중 print

`println`은 콘솔에 출력하는 내장 Scala 함수입니다. 생성된 회로는 Scala가 아닌 FIRRTL 또는 Verilog이기 때문에 회로 시뮬레이션 중에 인쇄하는 데 **사용할 수 없습니다**.

다음 코드 블록은 다양한 print 스타일을 보여줍니다.

In [10]:
class PrintingModule extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := io.in

    printf("Print during simulation: Input is %d\n", io.in)
    // chisel printf has its own string interpolator too
    printf(p"Print during simulation: IO is $io\n")

    println(s"Print during generation: Input is ${io.in}")
}

test(new PrintingModule ) { c =>
    c.io.in.poke(3.U)
    c.clock.step(5) // circuit will print
    
    println(s"Print during testing: Input is ${c.io.in.peek()}")
}

Elaborating design...
Print during generation: Input is UInt<4>(IO in unelaborated PrintingModule)
Done elaborating.
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during testing: Input is UInt<4>(3)
Print during simulation: Input is   0
Print during simulation: IO is AnonymousBundle(in ->   0, out ->   0)
test PrintingModule Success: 0 tests passed in 7 cycles in 0.003590 seconds 1949.67 Hz


defined class PrintingModule